《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》读书笔记。
 
 [第11章](/2017/07/24/python_data_analysis11.html)  第1节：数据规整化方面的话题

所有用到的数据可以从[作者的 github](https://github.com/wesm/pydata-book)下载。


In [1]:
%pylab inline
import pandas as pd
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


数据规整：Data munging


## 时间序列以及截面对齐

处理金融数据时，最费神的一个问题就是所谓的数据对齐（data alignment）。

两个时间序列的索引可能没有很好的对齐，或者两个DataFrame对象可能含有不匹配的行或者列。

MATLAB、R用户通常会耗费大量的时间来进行数据对对齐工作。

pandas可以在运算中自动对齐数据。这是极好的，会提高效率。

In [2]:
close_px = pd.read_csv('data/ch11/stock_px.csv', parse_dates=True, index_col=0)
volume = pd.read_csv('data/ch11/volume.csv', parse_dates=True, index_col=0)
prices = close_px.ix['2011-09-05':'2011-09-14', ['AAPL', 'JNJ', 'SPX', 'XOM']]
volume = volume.ix['2011-09-05':'2011-09-12', ['AAPL', 'JNJ', 'XOM']]

In [6]:
prices

,AAPL,JNJ,SPX,XOM
2011-09-06,379.74,64.64,1165.24,71.15
2011-09-07,383.93,65.43,1198.62,73.65
2011-09-08,384.14,64.95,1185.90,72.82
2011-09-09,377.48,63.64,1154.23,71.01
2011-09-12,379.94,63.59,1162.27,71.84
2011-09-13,384.62,63.61,1172.87,71.65
2011-09-14,389.30,63.73,1188.68,72.64


In [7]:
volume

,AAPL,JNJ,XOM
2011-09-06,18173500.0,15848300.0,25416300.0
2011-09-07,12492000.0,10759700.0,23108400.0
2011-09-08,14839800.0,15551500.0,22434800.0
2011-09-09,20171900.0,17008200.0,27969100.0
2011-09-12,16697300.0,13448200.0,26205800.0


In [10]:
# 如果想计算一个基于成交量的加权平均价
# pandas在算术运算时会自动对齐数据
# sum函数自动忽略NaN值
prices * volume

,AAPL,JNJ,SPX,XOM
2011-09-06,6.901205e+09,1.024434e+09,NaN,1.808370e+09
2011-09-07,4.796054e+09,7.040072e+08,NaN,1.701934e+09
2011-09-08,5.700561e+09,1.010070e+09,NaN,1.633702e+09
2011-09-09,7.614489e+09,1.082402e+09,NaN,1.986086e+09
2011-09-12,6.343972e+09,8.551710e+08,NaN,1.882625e+09
2011-09-13,NaN,NaN,NaN,NaN
2011-09-14,NaN,NaN,NaN,NaN


In [11]:
# 计算基于成交量的加权平均价
vwap = (prices * volume).sum() / volume.sum()

In [9]:
vwap

AAPL    380.655181
JNJ      64.394769
SPX            NaN
XOM      72.024288
dtype: float64

In [12]:
vwap.dropna()

AAPL    380.655181
JNJ      64.394769
XOM      72.024288
dtype: float64

In [13]:
# 如果需要手工对齐，可以使用DataFrame的align方法
prices.align(volume, join='inner')

(              AAPL    JNJ    XOM
 2011-09-06  379.74  64.64  71.15
 2011-09-07  383.93  65.43  73.65
 2011-09-08  384.14  64.95  72.82
 2011-09-09  377.48  63.64  71.01
 2011-09-12  379.94  63.59  71.84,
                   AAPL         JNJ         XOM
 2011-09-06  18173500.0  15848300.0  25416300.0
 2011-09-07  12492000.0  10759700.0  23108400.0
 2011-09-08  14839800.0  15551500.0  22434800.0
 2011-09-09  20171900.0  17008200.0  27969100.0
 2011-09-12  16697300.0  13448200.0  26205800.0)

In [14]:
# 通过一组索引可能不同的Series构建DataFrame
s1 = Series(range(3), index=['a', 'b', 'c'])
s2 = Series(range(4), index=['d', 'b', 'c', 'e'])
s3 = Series(range(3), index=['f', 'a', 'c'])
DataFrame({'one': s1, 'two': s2, 'three': s3})

,one,three,two
a,0.0,1.0,NaN
b,1.0,NaN,1.0
c,2.0,2.0,2.0
d,NaN,NaN,0.0
e,NaN,NaN,3.0
f,NaN,0.0,NaN


In [15]:
# 可以指定结果的索引（丢弃其余的数据）
DataFrame({'one': s1, 'two': s2, 'three': s3}, index=list('face'))

,one,three,two
f,NaN,0.0,NaN
a,0.0,1.0,NaN
c,2.0,2.0,2.0
e,NaN,NaN,3.0


## 频率不同的时间按序列的运算

经济学时间序列常常按年月日等频率进行数据统计。但有些是无规律的。

频率转换和重对齐的主要工具是resample 和 reindex 方法：

- resample 用于将数据转换到固定频率
- reindex 用于使数据符合一个新索引

二者都支持插值逻辑。


In [18]:
# 一个简单的周时间序列
ts1 = Series(np.random.randn(3),
             index=pd.date_range('2012-6-13', periods=3, freq='W-WED'))
ts1

2012-06-13   -0.928173
2012-06-20    0.506413
2012-06-27    1.052517
Freq: W-WED, dtype: float64

In [19]:
# 重采样到工作日，就会有缺省值出现
ts1.resample('B').mean()

2012-06-13   -0.928173
2012-06-14         NaN
2012-06-15         NaN
2012-06-18         NaN
2012-06-19         NaN
2012-06-20    0.506413
2012-06-21         NaN
2012-06-22         NaN
2012-06-25         NaN
2012-06-26         NaN
2012-06-27    1.052517
Freq: B, dtype: float64

In [23]:
# 用前面的值填充 NaN
ts1.resample('B').ffill()

2012-06-13   -0.928173
2012-06-14   -0.928173
2012-06-15   -0.928173
2012-06-18   -0.928173
2012-06-19   -0.928173
2012-06-20    0.506413
2012-06-21    0.506413
2012-06-22    0.506413
2012-06-25    0.506413
2012-06-26    0.506413
2012-06-27    1.052517
Freq: B, dtype: float64

In [24]:
# 更一般的不规则时间序列
dates = pd.DatetimeIndex(['2012-6-12', '2012-6-17', '2012-6-18',
                          '2012-6-21', '2012-6-22', '2012-6-29'])
ts2 = Series(np.random.randn(6), index=dates)
ts2

2012-06-12    0.131619
2012-06-17    1.440314
2012-06-18    0.780129
2012-06-21    1.024207
2012-06-22   -0.660424
2012-06-29   -0.218203
dtype: float64

In [26]:
# 如果想将处理过后的ts1加到ts2上，可以先将两个频率弄相同再相加
# 也可以用reindex方法，维持 ts2 的日期索引
ts1.reindex(ts2.index, method='ffill')

2012-06-12         NaN
2012-06-17   -0.928173
2012-06-18   -0.928173
2012-06-21    0.506413
2012-06-22    0.506413
2012-06-29    1.052517
dtype: float64

In [27]:
ts2 + ts1.reindex(ts2.index, method='ffill')

2012-06-12         NaN
2012-06-17    0.512141
2012-06-18   -0.148044
2012-06-21    1.530620
2012-06-22   -0.154011
2012-06-29    0.834314
dtype: float64

### 使用 Period

Period 提供了另一种处理不同频率时间序列的方法。

In [28]:
# 比如，一个公司可能会发布其以6月结尾的财年的每季度盈利报告，即频率为Q-JUN
gdp = Series([1.78, 1.94, 2.08, 2.01, 2.15, 2.31, 2.46],
             index=pd.period_range('1984Q2', periods=7, freq='Q-SEP'))
infl = Series([0.025, 0.045, 0.037, 0.04],
              index=pd.period_range('1982', periods=4, freq='A-DEC'))
gdp

1984Q2    1.78
1984Q3    1.94
1984Q4    2.08
1985Q1    2.01
1985Q2    2.15
1985Q3    2.31
1985Q4    2.46
Freq: Q-SEP, dtype: float64

In [29]:
infl

1982    0.025
1983    0.045
1984    0.037
1985    0.040
Freq: A-DEC, dtype: float64

In [30]:
# 与Timestamp的时间序列不同，由period索引的不同频率的时间序列之间的运算必须进行显示转换
# 假设已知 infl 值是每年年末观测的，于是可以将其转换为 Q-SEP ，以得到改频率下的正确时期
infl_q = infl.asfreq('Q-SEP', how='end')

In [31]:
infl_q

1983Q1    0.025
1984Q1    0.045
1985Q1    0.037
1986Q1    0.040
Freq: Q-SEP, dtype: float64

In [32]:
# 显示转换以后，就可以被重新索引了（使用向前填充ffill ,以匹配gdp）
infl_q.reindex(gdp.index, method='ffill')

1984Q2    0.045
1984Q3    0.045
1984Q4    0.045
1985Q1    0.037
1985Q2    0.037
1985Q3    0.037
1985Q4    0.037
Freq: Q-SEP, dtype: float64

### 时间和“最当前”数据选取

假设有一个很长的盘中数据，希望抽取其中每天特定时间的价格数据。如果数据不规整该怎么办？

In [34]:
# Make an intraday date range and time series
rng = pd.date_range('2012-06-01 09:30', '2012-06-01 15:59', freq='T')
# Make a 5-day series of 9:30-15:59 values
rng = rng.append([rng + pd.offsets.BDay(i) for i in range(1, 4)])
ts = Series(np.arange(len(rng), dtype=float), index=rng)
ts.head()

2012-06-01 09:30:00    0.0
2012-06-01 09:31:00    1.0
2012-06-01 09:32:00    2.0
2012-06-01 09:33:00    3.0
2012-06-01 09:34:00    4.0
dtype: float64

In [37]:
# 只取10点钟的数据
from datetime import time
ts[time(10, 0)]

2012-06-01 10:00:00      30.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1200.0
dtype: float64

In [38]:
# 该操作实际上用了实例方法at_time（各时间序列以及类似的DataFrame对象都有）
ts.at_time(time(10, 0))

2012-06-01 10:00:00      30.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1200.0
dtype: float64

In [39]:
# 选取两个Time对象之间的值
ts.between_time(time(10, 0), time(10, 1))

2012-06-01 10:00:00      30.0
2012-06-01 10:01:00      31.0
2012-06-04 10:00:00     420.0
2012-06-04 10:01:00     421.0
2012-06-05 10:00:00     810.0
2012-06-05 10:01:00     811.0
2012-06-06 10:00:00    1200.0
2012-06-06 10:01:00    1201.0
dtype: float64

In [40]:
# 可能刚好就没有任何数据落在某个具体的时间上（比如上午10点）。这时，可能会希望得到上午10点之前最后出现的值
#下面将该时间序列的大部分内容随机设置为NA
np.random.seed(12346)
indexer = np.sort(np.random.permutation(len(ts))[700:])
irr_ts = ts.copy()
irr_ts[indexer] = np.nan
irr_ts['2012-06-01 09:50':'2012-06-01 10:00']

2012-06-01 09:50:00    20.0
2012-06-01 09:51:00     NaN
2012-06-01 09:52:00    22.0
2012-06-01 09:53:00    23.0
2012-06-01 09:54:00     NaN
2012-06-01 09:55:00    25.0
2012-06-01 09:56:00     NaN
2012-06-01 09:57:00     NaN
2012-06-01 09:58:00     NaN
2012-06-01 09:59:00     NaN
2012-06-01 10:00:00     NaN
dtype: float64

In [41]:
#如果将一组Timestamp传入asof方法，就能得到这些时间点处（或其之前最近）的有效值（非NA）。
# 例如，构造一个日期范围（每天上午10点），然后将其传入asof
selection = pd.date_range('2012-06-01 10:00', periods=4, freq='B')
irr_ts.asof(selection)

2012-06-01 10:00:00      25.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1197.0
Freq: B, dtype: float64

## 拼接多个数据源

在第七章中曾经介绍了数据拼接的知识，在金融或经济中，还有另外几个经常出现的情况：

- 在一个特定的时间点上，从一个数据源切换到另一个数据源
- 用另一个时间序列对当前时间序列中的缺失值“打补丁”
- 将数据中的符号（国家、资产代码等）替换为实际数据

In [42]:
# 关于特定时间的数据源切换，就是用concat函数进行连接
data1 = DataFrame(np.ones((6, 3), dtype=float),
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/12/2012', periods=6))
data2 = DataFrame(np.ones((6, 3), dtype=float) * 2,
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/13/2012', periods=6))
spliced = pd.concat([data1.ix[:'2012-06-14'], data2.ix['2012-06-15':]])
spliced

,a,b,c
2012-06-12,1.0,1.0,1.0
2012-06-13,1.0,1.0,1.0
2012-06-14,1.0,1.0,1.0
2012-06-15,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0


In [43]:
# 假设data1缺失了data2中存在的某个时间序列
data2 = DataFrame(np.ones((6, 4), dtype=float) * 2,
                  columns=['a', 'b', 'c', 'd'],
                  index=pd.date_range('6/13/2012', periods=6))
spliced = pd.concat([data1.ix[:'2012-06-14'], data2.ix['2012-06-15':]])
spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,NaN
2012-06-14,1.0,1.0,1.0,NaN
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


In [44]:
# combine_first可以引入合并点之前的数据，这样也就扩展了'd'项的历史
spliced_filled = spliced.combine_first(data2)
spliced_filled

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


In [46]:
# DataFrame也有一个类似的方法update，它可以实现就地更新
# 如果只想填充空洞，则必须差U纳入overwrite = False才行
# 不传入overwrite会把整条数据都覆盖

spliced.update(data2, overwrite=False)
spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


In [47]:
# 上面所讲的技术可以将数据中的符号替换为实际数据
# 但有时利用 DataFrame的索引机制直接进行设置会更简单一些
cp_spliced = spliced.copy()
cp_spliced[['a', 'c']] = data1[['a', 'c']]
cp_spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,1.0,2.0,1.0,2.0
2012-06-16,1.0,2.0,1.0,2.0
2012-06-17,1.0,2.0,1.0,2.0
2012-06-18,NaN,2.0,NaN,2.0


## 收益指数和累计收益

金融领域中，收益（return）通常指的是某资产价格的百分比变化。

In [69]:
# 2011到2012年苹果公司的股票价格数据
from pandas_datareader import data, yahoo
price =yahoo.daily.YahooDailyReader.read('AAPL')['Adj Close']
price[-5:]

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
# 计算两个时间点之间的累计百分比回报只需计算价格的百分比变化即可
price['2011-10-03'] / price['2011-3-01'] - 1

In [ ]:
# 通常会先算出一个收益指数，它表示单位投资（比如1美元）收益的时间序列
# 从收益指数中可以得出许多假设。例如，人们可以决定是否进行利润再投资
# 可以用cumprod计算出一个简单的收益指数

returns = price.pct_change()
ret_index = (1 + returns).cumprod()
ret_index[0] = 1  # Set first value to 1
ret_index

In [70]:
# 得到收益指数之后，计算指定时期内的累计收益就很简单了
m_returns = ret_index.resample('BM', how='last').pct_change()
m_returns['2012']

NameError: name 'ret_index' is not defined

In [71]:
#如果知道了股息的派发日和支付率，就可以将它们计入到每日总收益中
m_rets = (1 + returns).resample('M', how='prod', kind='period') - 1
m_rets['2012']

NameError: name 'returns' is not defined

In [72]:
returns[dividend_dates] += dividend_pcts

NameError: name 'returns' is not defined